<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Subqueries
              
</p>
</div>

Data Science Cohort Live NYC Feb 2022
<p>Phase 1: Topic 6</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

Queries can get complex:
- Helpful/necessary to build a query out of sub-units
- These subunits are queries which might include:
    - groupbys/agg functions
    - selecting/filtering
    - etc.

<img src = "Images/sql_subquery.jpg">

Inner query is a **subquery**:

- Generates an intermediate table that stores results temporarily in memory.
- Subquery intermediate table can be used during execution of main query.

Subqueries can be used to great effect in a variety of circumstances:
    
- Selecting from a table created by a subquery 
    - Subquery in FROM statement
- Filtering on a subquery.
    - Subquery in a WHERE statement
- Filtering in a SELECT statement.


#### Subquery in FROM clause

First load our handy dandy flights database.

In [55]:
import sqlite3
import numpy as np
import pandas as pd

# connect to flights database
con = sqlite3.connect('data/flights.db')

Recall the various tables in flight database:

In [56]:
%%bash 

sqlite3 data/flights.db
.tables

airlines  airports  routes  


In [57]:
pd.read_sql("""

SELECT * 
FROM airports
LIMIT 2

""", con)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


#### Problem

Get the average altitude of the highest altitude airports in the United States, Russia, and China.

Useful to break this into steps with a subquery.

- First filter airports on country.
- Then group by country and aggregate.
- Get average of the altitudes of the resultant airports.

Step by step.

Can also control execution flow and number of operations in a query this way.

Step 1: Filter airports on relevant countries.

In [59]:
sq1 = pd.read_sql("""

SELECT *

FROM

airports

WHERE country IN 
('United States', 'China', 'Russia')
     

""", con)
sq1

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,318,6891,Putnam County Airport,Greencastle,United States,4I7,\N,39.6335556,-86.8138056,842,-5,U,America/New_York
1,1104,6890,Dowagiac Municipal Airport,Dowagiac,United States,C91,\N,41.9929342,-86.1280125,748,-5,U,America/New_York
2,1121,6889,Cambridge Municipal Airport,Cambridge,United States,CDI,\N,39.9750278,-81.5775833,799,-5,U,America/New_York
3,1470,6885,Door County Cherryland Airport,Sturgeon Bay,United States,SUE,\N,44.8436667,-87.4215556,725,-6,U,America/Chicago
4,1507,6884,Shoestring Aviation Airfield,Stewartstown,United States,0P2,\N,39.7948244,-76.6471914,1000,-5,U,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,8079,9514,Ferry Terminal,Valdez,United States,None,VLDZ,61.123976,-146.365309,0,-9,A,America/Anchorage
2161,8090,9525,Ferry Dock,Bainbridge Island,United States,None,BAIN,47.622237,-122.509362,0,-8,A,America/Los_Angeles
2162,8098,9533,Chatsworth Station,Chatsworth,United States,CWT,\N,34.256944,-118.598889,978,-8,A,America/Los_Angeles
2163,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


In [35]:
pd.read_sql("""

SELECT * 
FROM airports
LIMIT 2

""", con)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


In [36]:
sq1

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,318,6891,Putnam County Airport,Greencastle,United States,4I7,\N,39.6335556,-86.8138056,842,-5,U,America/New_York
1,1104,6890,Dowagiac Municipal Airport,Dowagiac,United States,C91,\N,41.9929342,-86.1280125,748,-5,U,America/New_York
2,1121,6889,Cambridge Municipal Airport,Cambridge,United States,CDI,\N,39.9750278,-81.5775833,799,-5,U,America/New_York
3,1470,6885,Door County Cherryland Airport,Sturgeon Bay,United States,SUE,\N,44.8436667,-87.4215556,725,-6,U,America/Chicago
4,1507,6884,Shoestring Aviation Airfield,Stewartstown,United States,0P2,\N,39.7948244,-76.6471914,1000,-5,U,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,8079,9514,Ferry Terminal,Valdez,United States,None,VLDZ,61.123976,-146.365309,0,-9,A,America/Anchorage
2161,8090,9525,Ferry Dock,Bainbridge Island,United States,None,BAIN,47.622237,-122.509362,0,-8,A,America/Los_Angeles
2162,8098,9533,Chatsworth Station,Chatsworth,United States,CWT,\N,34.256944,-118.598889,978,-8,A,America/Los_Angeles
2163,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


Step 2: Use filtered table (via subquery) and get max altitude airport by country 

Want: name, city, altitude of these airports

In [37]:
sq2 = pd.read_sql("""

SELECT cf.name, cf.city, 
MAX(CAST(altitude as INT)) AS maxalt
     
FROM 

(SELECT *
FROM airports 
WHERE country IN 
("United States", "Russia", "China") ) AS cf

GROUP BY country

     

""", con)

- Subquery in parentheses for defining temporary table
- Aliasing subquery: 
    - good practice
    - ease of referencing subquery fields in main query

In [38]:
sq2

,name,city,maxalt
0,Yading Daocheng,Daocheng,14472
1,Irkutsk-2,Irkutsk,13411
2,Silverton,Silverton,9308


Now get the mean altitude across the three countries:

In [60]:
sq3 = pd.read_sql("""

SELECT AVG(max_altitude) 
AS max_altitude_avg
    
FROM 

(
SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) 
     AS max_altitude

FROM 

(SELECT *
FROM airports 
WHERE country IN 
("United States", "Russia", "China") ) AS cf
GROUP BY cf.country
)

     

""", con)

In [40]:
sq3

,max_altitude_avg
0,12397.0


Two levels of subqueries gets the job done.

**Exercise**

- Get me the countries whose max altitude and min altitude airport have a differential greater than 2000 ft.

- Use a subquery in the FROM statement.

In [ ]:
sq3 = pd.read_sql("""



""", con)
sq3

#### Using derived tables in joins

- create set of tables from subqueries
- use these to create a new table with derived columns from subqueries. 

Let's take an example of this.

For each airport I want:
- the count of routes that leaves from an airport.
- the count of routes that arrive there.


Many ways to do this. But I'll make full use of subqueries and joins here.

In [42]:
pd.read_sql("""

SELECT * 
     FROM routes
     LIMIT 5


""", con)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


First calculate count of routes departing from each airport.


In [61]:
departure_table = pd.read_sql("""

SELECT source, COUNT(*) as departure_count

FROM routes

GROUP BY source

     
""", con)

In [62]:
departure_table = pd.read_sql("""

SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source
     
""", con)

In [45]:
departure_table

,source,departure_count
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


In [63]:
arrival_table = pd.read_sql("""

SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest
     
""", con)

In [64]:
arrival_table

,dest,arrival_count
0,AAE,9
1,AAL,21
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3413,ZUH,59
3414,ZUM,2
3415,ZVK,3
3416,ZYI,15


We can link these the arrival tables and destination tables together:
- Keep records where source and destination are in both tables.
- What operation?

INNER JOIN ON SUBQUERY RESULTS!

In [66]:
%%capture full_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [27]:
full_result()

,airport_code,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


This result is a derived table telling us information for each airport and keeping relevant records.

- We can filter or aggregate on this derived table as we see fit.


- Get all airports with significance difference in:
    - number of routes originating and terminating at given airport.
    - dfference in departure and arrival route counts greater than 5.
- Order in descending order of this difference.

Help this hapless data science instructor out!

In [67]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest


     
""", con)

In [68]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count, 
        ABS(dep.departure_count - arrival_count) AS count_diff
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest

WHERE count_diff > 5
ORDER BY count_diff DESC
     
""", con)

In [69]:
filtered_result()

,airport_code,departure_count,arrival_count,count_diff
0,AKL,96,117,21
1,MED,39,59,20
2,PWM,2,18,16
3,DEN,361,374,13
4,JED,194,183,11
5,ATH,197,206,9
6,CRW,6,15,9
7,HOU,79,70,9
8,YHZ,43,52,9
9,BNE,152,144,8


**Exercise**

Get me the name, city, country and airport code of airports for countries with more than 10 active airlines companies

Hints:
- DISTINCT(name) will be useful.
- A join might be useful. Aliasing tables will be important here as well.
- Think of groupbys and aggregations and filtering on them.

In [6]:
%%bash 

sqlite3 data/flights.db
.schema airlines

CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
);
CREATE INDEX ix_airlines_index ON airlines ([index]);


In [26]:
%%bash 

sqlite3 data/flights.db
.schema airports

CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
);
CREATE INDEX ix_airports_index ON airports ([index]);


In [71]:
# Get me the name, city, country and airport code of airports for countries 
# with more than 10 active airlines companies

# Let's have fun building this up. Step by step. Start with subquery.
pd.read_sql("""
SELECT ar.name, ar.code, ar.city, ar.country, ct
FROM 
( SELECT country, COUNT(DISTINCT name) AS ct
FROM airlines 
WHERE active = 'Y'
GROUP BY country
HAVING ct > 10 ) as inter INNER JOIN airports as ar
ON inter.country = ar.country

""", con)

,name,code,city,country,ct
0,Sault Ste Marie,YAM,Sault Sainte Marie,Canada,34
1,Winnipeg St Andrews,YAV,Winnipeg,Canada,34
2,Shearwater,YAW,Halifax,Canada,34
3,St Anthony,YAY,St. Anthony,Canada,34
4,Tofino,YAZ,Tofino,Canada,34
...,...,...,...,...,...
4984,Mansons Landing Water Aerodrome,YMU,Mansons Landing,Canada,34
4985,Port McNeill Airport,YMP,Port McNeill,Canada,34
4986,Sullivan Bay Water Aerodrome,YTG,Sullivan Bay,Canada,34
4987,Deer Harbor Seaplane,DHB,Deer Harbor,United States,141


- Complex as subqueries pile on. 
- There are ways to organize subqueries: common table expressions
- Will make code readable.

But first: other ways subqueries can be used.

#### Subqueries in the WHERE statement

- Can use the result of a subquery to filter another selection.
- Particularly useful when you want to filter on the results of:
    - groupbys, etc
    - results from other tables

I want all route records where the departure airport has a departure route count greater than 200.

In [32]:

pd.read_sql('''


SELECT source, COUNT(source)
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200
''', con)

,source,COUNT(source)
0,AGP,206
1,AMS,453
2,ARN,219
3,ATL,915
4,AUH,241
...,...,...
63,VIE,308
64,XIY,283
65,XMN,233
66,YYZ,319


In [73]:
%%capture where_subquery1
pd.read_sql('''


SELECT *
            
FROM routes

WHERE source IN (SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200) 
''', con)

In [74]:
where_subquery1()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,5,2B,410,DME,4029,KZN,2990,None,0,CR2
1,6,2B,410,DME,4029,NBC,6969,None,0,CR2
2,7,2B,410,DME,4029,TGK,\N,None,0,CR2
3,8,2B,410,DME,4029,UUA,6160,None,0,CR2
4,123,2L,2750,ZRH,1678,BDS,1506,None,0,100
...,...,...,...,...,...,...,...,...,...,...
21698,67646,ZL,4178,SYD,3361,OAG,6793,None,0,SF3
21699,67647,ZL,4178,SYD,3361,PKE,6317,None,0,SF3
21700,67648,ZL,4178,SYD,3361,TRO,6794,None,0,SF3
21701,67649,ZL,4178,SYD,3361,WGA,3363,None,0,SF3


Subqueries made this selection possible. 


#### The Semijoin

- Using a subquery from one table to filter another table.

Example:

Get the airport name, code, country, and altitude for airports with more than 200 routes originating from that airport.

In [75]:
%%capture semijoin
pd.read_sql('''

SELECT name, code, country, 
CAST(altitude AS int) AS altitde
FROM airports

WHERE code IN (
SELECT source
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200 
)
''', con)

In [76]:
semijoin()

,name,code,country,altitde
0,Lester B Pearson Intl,YYZ,Canada,569
1,Brussels Natl,BRU,Belgium,184
2,Frankfurt Main,FRA,Germany,364
3,Dusseldorf,DUS,Germany,147
4,Franz Josef Strauss,MUC,Germany,1487
...,...,...,...,...
63,Mc Carran Intl,LAS,United States,2141
64,Orlando Intl,MCO,United States,96
65,Suvarnabhumi Intl,BKK,Thailand,5
66,Son Sant Joan,PMI,Spain,24


**Exercise**

Get me routes (all columns) whose destination airport is in the America/New York time zone.

In [79]:
%%bash

sqlite3 data/flights.db
.schema airports

CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
);
CREATE INDEX ix_airports_index ON airports ([index]);


In [80]:
%%bash

sqlite3 data/flights.db
.schema airlines

CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
);
CREATE INDEX ix_airlines_index ON airlines ([index]);


In [88]:
# help me out 
pd.read_sql('''
SELECT *
FROM routes
WHERE dest IN
(SELECT code 
FROM airports 
WHERE timezone = 'America/New_York')

''', con)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,443,3M,\N,ATL,3682,LWB,6958,None,0,SF3
1,444,3M,\N,ATL,3682,MCN,3754,None,0,SF3
2,449,3M,\N,BIM,1937,FLL,3533,None,0,SF3
3,450,3M,\N,ELH,1943,FLL,3533,None,0,SF3
4,451,3M,\N,ELH,1943,PBI,3722,None,0,SF3
...,...,...,...,...,...,...,...,...,...,...
6154,65395,X7,1775,MIA,3576,APF,4012,None,0,CNA
6155,65441,XL,1034,GYE,2673,JFK,3797,None,0,763
6156,65442,XL,1034,GYE,2673,MIA,3576,Y,0,738 763 757
6157,65451,XL,1034,UIO,2688,MIA,3576,None,0,763


#### The antijoin

Just the negation of the semijoin:

In [89]:
%%capture antijoin

pd.read_sql('''
SELECT *
FROM airports

WHERE NOT code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 300 
)
''', con)

In [90]:
antijoin()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5843,8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8,A,America/Vancouver
5844,8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8,A,America/Vancouver
5845,8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8,A,America/Vancouver
5846,8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.00596,0,-8,A,America/Los_Angeles


#### Subqueries in the SELECT statement

Get altitude of airport side by side with average airport altitude in the respective country.

In [91]:
%%capture select_subquery

pd.read_sql('''
SELECT a1.country,
name, 
CAST(altitude as int) AS altitude ,

(SELECT AVG(CAST(altitude as int))
FROM airports AS a2
WHERE a2.country == a1.country
) AS avg_alt

FROM airports AS a1


''', con)

Sometimes known as a **correlated subquery**.

- Inner select statement filters a2 by country matching a given row's country in a1. Takes average altitude.

- Does this for each row in a1. The filtering of a2 by country *correlated* with current row in a1.


**Correlated subqueries can be a bit slow**


In [92]:
select_subquery()

,country,name,altitude,avg_alt
0,Papua New Guinea,Goroka,5282,852.982456
1,Papua New Guinea,Madang,20,852.982456
2,Papua New Guinea,Mount Hagen,5388,852.982456
3,Papua New Guinea,Nadzab,239,852.982456
4,Papua New Guinea,Port Moresby Jacksons Intl,146,852.982456
...,...,...,...,...
8102,Canada,Mansons Landing Water Aerodrome,0,773.326437
8103,Canada,Port McNeill Airport,225,773.326437
8104,Canada,Sullivan Bay Water Aerodrome,0,773.326437
8105,United States,Deer Harbor Seaplane,0,951.795522


#### Common Table Expressions (CTEs)

Useful for when:

- Multiple subqueries floating around.
- Reusing same subqueries over and over again in same statement.

Cleans code up, reduces error, makes readable.

CTE syntax:

WITH subquery1_name AS (subquery1 statement),
<br>
WITH subquery2_name AS (subquery2 statement), 
<br>
...., 
<br>
WITH subquery_n_name AS (subquery_n statement)
<br>
<br>
MAIN QUERY

In [93]:
%%capture filtered_result

pd.read_sql("""

WITH s1 AS (
     SELECT source, COUNT(source) as departure_count
     FROM routes
     GROUP BY source),
     
     s2 AS ( 
     SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     GROUP BY dest
     )


SELECT source, departure_count, arrival_count
FROM
s1 INNER JOIN s2
ON s1.source = s2.dest

     
""", con)

In [94]:
filtered_result()

,source,departure_count,arrival_count
0,AAE,9,9
1,AAL,20,21
2,AAN,2,2
3,AAQ,3,3
4,AAR,8,8
...,...,...,...
3397,ZUH,60,59
3398,ZUM,2,2
3399,ZVK,3,3
3400,ZYI,15,15


CTEs make the primary query easy to read.

- Subqueries can just be referenced by looking it up in the CTE.

In [137]:
df3 = pd.read_sql("""

SELECT source, dest,
ap1.city AS source_city,
ap2.city AS dest_city,
ap1.country AS source_country, 
ap2.country AS dest_country,
CAST(ap1.altitude as float) AS alt,
max_alt

FROM routes
INNER JOIN 

(
SELECT * 
FROM
airports
WHERE country == 'United States') as ap1


INNER JOIN 

(
SELECT country, 
city, name, code, 
MAX(CAST(altitude as float)) AS max_alt

FROM airports 
GROUP BY country
)  as ap2

ON (routes.source == ap1.code AND routes.dest == ap2.code)

""", con)